In [1]:
import csv
import numpy as np
from collections import Counter
from nltk.corpus import brown
from mittens import GloVe, Mittens
# from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.notebook import tqdm
import pickle

from LanguageModels.Word2Vec import Word2Vec
from Preprocessing.LemmatizerPreprocessor import LemmatizerPreprocessor
from Preprocessing.DataLoader import DataLoader

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ryan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ryan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ryan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ryan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# Load data
data = DataLoader('../data/EMNLP2020.csv').load()

# Load preprocessor
lp = LemmatizerPreprocessor()

clean_abstracts = []
words = []
for row in data:
    abstract = row['abstract']
    abstract_clean = lp.clean(abstract)
    words.extend(abstract_clean)
    clean_abstracts.append(' '.join(abstract_clean))
    
words = set(words)

In [3]:
def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

def get_rareoov(xdict, val):
    return [k for (k,v) in Counter(xdict).items() if v<=val]

In [20]:
dim = 200
glove_path = "../data/glove.6B/glove.6B.%dd.txt" % dim
pre_glove = glove2dict(glove_path)

In [9]:
oov = [token for token in words if token not in pre_glove.keys()]
print(len(oov))

1446


In [10]:
corp_vocab = list(set(oov))
brown_doc = [' '.join(words)]

cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
X = cv.fit_transform(brown_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

In [11]:
mittens_model = Mittens(n=dim, max_iter=1000)

new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=corp_vocab,
    initial_embedding_dict= pre_glove)

Iteration 1000: error 0.0061

In [22]:
oov_glove = dict(zip(corp_vocab, new_embeddings))
finetuned = {}
finetuned.update(oov_glove)
finetuned.update(pre_glove)

print(len(oov_glove))
print(len(pre_glove))
print(len(finetuned))

f = open("../data/glove.6B/glove.6B.%dd.finetuned.p" % dim,"wb")
pickle.dump(finetuned, f)
f.close()

1446
400000
401446


In [50]:
# newglove = dict(zip(corp_vocab, new_embeddings))
